In [ ]:
import argparse
import json
import tensorboard
import tensorboardX
import os
import argparse
import json
import os
import numpy as np
import torch
import torch.nn as nn
import nni
from nni.nas.nn.pytorch import ModelSpace, LayerChoice, MutableConv2d, MutableBatchNorm2d, MutableReLU
from pytorch_lightning import Trainer
from nni.nas.evaluator.pytorch import Lightning, ClassificationModule, Trainer
from nni.nas.experiment import NasExperiment
from nni.nas.space import model_context
from nni.nas.hub.pytorch import DARTS
from nni.nas.strategy import DARTS as DartsStrategy
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms
from torchvision.datasets import CIFAR10
from nni.nas.experiment import NasExperiment
from nni.nas.evaluator import FunctionalEvaluator
from nni.nas.evaluator import FunctionalEvaluator
import nni.nas.strategy as strategy
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
#from ops import AvgPool,DilConv,SepConv
import genotypes
from pytorch_lightning.callbacks import ModelCheckpoint
torch.set_float32_matmul_precision('medium')
from tqdm import tqdm
from nni.nas.nn.pytorch import LayerChoice, ModelSpace,ValueChoice
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from pytorch_lightning import LightningModule, Trainer
from torchvision import datasets, transforms
from nni.nas.evaluator.pytorch import Classification

In [ ]:
@nni.trace
class AuxLossClassificationModule(ClassificationModule):
    """Several customization for the training of DARTS, based on default Classification."""
    model: DARTS
    def __init__(self,
                 learning_rate: float = 1e-5,
                 weight_decay: float = 0.,
                 auxiliary_loss_weight: float = 0.4,
                 max_epochs: int = 600):
        print(f"lr : {learning_rate}")
        print(f"weight decay: {weight_decay}")
        print(f"aux loss weight: {auxiliary_loss_weight}")
        print(f"max epochs: {max_epochs}")
        super().__init__(learning_rate=learning_rate, weight_decay=weight_decay, num_classes=10)
        self.auxiliary_loss_weight = auxiliary_loss_weight
        self.max_epochs = max_epochs
        self.criterion=  nn.CrossEntropyLoss()

    def configure_optimizers(self):
        """Customized optimizer with momentum, as well as a scheduler."""
        """
        Classification Module params from nni.nas.evaluators
             learning_rate: float
             weight_decay: float
             optimizer: Type[optim.Optimizer]
             export_onnx: bool 
             num_classes: Optional[int] 

        """
        optimizer = torch.optim.Adam(
            self.parameters(),
            lr = 1e-6,
            betas=(0.9, 0.999),  # type: ignore
            eps=1e-07,
            weight_decay= self.auxiliary_loss_weight  # type: ignore
        )
        return {
            'optimizer': optimizer,
            'lr_scheduler': torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, self.max_epochs, eta_min=1e-3)
        }

    def training_step(self, batch, batch_idx):
        """Training step, customized with auxiliary loss."""
        x, y = batch

        # Check for NaNs or infinite values in input
        if torch.isnan(x).any() or torch.isnan(y).any() or torch.isinf(x).any() or torch.isinf(y).any():
            raise ValueError("Input data contains NaNs or Infinities.")

        if self.auxiliary_loss_weight:
            y_hat, y_aux = self(x)
            loss_main = self.criterion(y_hat, y)
            loss_aux = self.criterion(y_aux, y)
            # Check for NaNs in loss values
            if torch.isnan(loss_main).any() or torch.isnan(loss_aux).any():
                raise ValueError("Loss contains NaNs.")
            self.log('train_loss_main', loss_main)
            self.log('train_loss_aux', loss_aux)
            loss = loss_main + self.auxiliary_loss_weight * loss_aux

        else:
            y_hat = self(x)
            loss = self.criterion(y_hat, y)       
        #acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log('train_loss', loss, prog_bar=True)
        #self.log('train_accuracy', acc, prog_bar=True)  # Log training accuracy
        for name, metric in self.metrics.items():
            self.log('train_' + name, metric(y_hat, y), prog_bar=True)
        return loss
        



    def on_train_epoch_start(self):
        """Set drop path probability before every epoch. This has no effect if drop path is not enabled in model."""
        self.model.set_drop_path_prob(self.model.drop_path_prob * self.current_epoch / self.max_epochs)

        # Logging learning rate at the beginning of every epoch
        self.log('lr', self.trainer.optimizers[0].param_groups[0]['lr'])




In [ ]:
checkpoint_path = './checkpoints/best-checkpoint-v1.ckpt'

checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))

model = AuxLossClassificationModule()

model_state_dict = model.state_dict()
pretrained_state_dict = checkpoint['state_dict']

filtered_state_dict = {k: v for k, v in pretrained_state_dict.items() if k in model_state_dict}

model_state_dict.update(filtered_state_dict)

model.load_state_dict(model_state_dict)

print(model)

# Print additional information from the checkpoint
if 'epoch' in checkpoint:
    print("Epoch:", checkpoint['epoch'])

if 'global_step' in checkpoint:
    print("Global Step:", checkpoint['global_step'])

if 'callbacks' in checkpoint and isinstance(checkpoint['callbacks'], dict):
    for key, callback in checkpoint['callbacks'].items():
        if isinstance(callback, dict) and 'best_model_score' in callback:
            print("Best Model Score (Train Accuracy):", callback['best_model_score'].item())